In [ ]:
import json
import pandas as pd
import numpy
import numpy as np
import holoviews as hv
import datetime
%matplotlib inline
hv.notebook_extension()

In [ ]:
with open('job_log_2016-11.json', 'r') as f:
    x = json.load(f)

data = []
for key, val in x.items():
    ID, user, process = key.split()
    for i in val:
        current_time, running_time, num_cores = i
        data.append(dict(ID=ID, user=user, process=process, current_time=current_time, 
                    running_time=running_time, num_cores=num_cores))

df = pd.DataFrame(data)
by_ID = df.groupby('ID')
df['elapsed_time'] = by_ID['current_time'].transform(lambda x: x-x.min())
df['running_time'] = by_ID['running_time'].transform(lambda x: x-x.min())
df['running_time'] = df['running_time'] / df['num_cores']
df.index = pd.to_datetime(df.pop('current_time'), unit='s', utc=True)

last_entries = pd.DataFrame([dict(group.iloc[-1]) for i, group in by_ID])
last_entries['percentage'] = last_entries['running_time'] / last_entries['elapsed_time']

In [ ]:
usage = []
for user, group in last_entries.groupby('user'):
    elapsed_days = (group['elapsed_time'] * group['num_cores']).sum() / 86400
    running_time_days = (group['running_time'] * group['num_cores']).sum() / 86400
    if running_time_days > 0.0:
        percentage = running_time_days / elapsed_days * 100
        usage.append(dict(user=user, elapsed_days=elapsed_days,
                          running_time_days=running_time_days, percentage=percentage))
df2 = pd.DataFrame(usage)

# Sorted on activity

In [ ]:
for i, row in df2.sort_values('percentage').iterrows():
    print('{} has {:.0f} days of computing and {:.1f}% time activity'.format(
            row.user, row.elapsed_days, row.percentage))

# Sorted on days

In [ ]:
for i, row in df2.sort_values('elapsed_days', ascending=False).iterrows():
    print('{} has {:.0f} days of computing and {:.1f}% time activity'.format(
            row.user, row.elapsed_days, row.percentage))

In [ ]:
df2.sort_values('percentage').plot.bar('user', 'percentage')

In [ ]:
df2.sort_values('elapsed_days').plot.bar('user', 'elapsed_days')

In [ ]:
df2.sort_values('running_time_days').plot.bar('user', 'running_time_days')